In [1]:
import os
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
!pip install xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import roc_auc_score

In [10]:
folder_path = 'model_folder'

labelEncoders=joblib.load(os.path.join(folder_path, 'labelEncoders.pkl'))
target_encoder=joblib.load(os.path.join(folder_path, 'target_encoder.pkl'))
sc_multiclass=joblib.load(os.path.join(folder_path, 'sc_multiclass.pkl'))
sc_numerical=joblib.load(os.path.join(folder_path, 'sc_numerical.pkl'))
numerical=joblib.load(os.path.join(folder_path, 'numerical.pkl'))
multiclass_cols=joblib.load(os.path.join(folder_path, 'multiclass_cols.pkl'))
binary_cols_from_cat_cols=joblib.load(os.path.join(folder_path, 'binary_cols_from_cat_cols.pkl'))
binary=joblib.load(os.path.join(folder_path, 'binary.pkl'))
xgb_model=joblib.load(os.path.join(folder_path, 'xgb_model.pkl'))
selector=joblib.load(os.path.join(folder_path, 'selector.pk1'))
dependent=joblib.load(os.path.join(folder_path, 'dependent.pk1'))

In [11]:
propensify_test_orig = pd.read_excel('test.xlsx')
propensify_test = propensify_test_orig.copy(deep=True)
multiclass_features_test = pd.DataFrame(target_encoder.transform(propensify_test[multiclass_cols]), columns = multiclass_cols)


binary_cols_from_cat_cols_new = binary_cols_from_cat_cols.copy()
binary_cols_from_cat_cols_new.remove(dependent)
binaryFromCat_test = propensify_test[binary_cols_from_cat_cols_new]
i=0
for col in binary_cols_from_cat_cols:
    if col!=dependent:
        encoder = labelEncoders[i]
        binaryFromCat_test[col] = encoder.transform(binaryFromCat_test[col])
    i += 1
    
multiclass_features_test = pd.DataFrame(sc_multiclass.transform(multiclass_features_test), columns = multiclass_cols)
numerical_features_test = pd.DataFrame(sc_numerical.transform(propensify_test[numerical]), columns = numerical)

binary_features_test = propensify_test[binary]

numerical_features_test.reset_index(drop=True, inplace=True)
binaryFromCat_test.reset_index(drop=True, inplace=True)
multiclass_features_test.reset_index(drop=True, inplace=True)
binary_features_test.reset_index(drop=True, inplace=True)

p_test = pd.concat([numerical_features_test,binary_features_test,binaryFromCat_test, multiclass_features_test], axis=1)

X_test_reducedDim = p_test.loc[:,selector.support_]

test_dm = xgb.DMatrix(data =  X_test_reducedDim, feature_names=list(X_test_reducedDim.columns))
y_test_pred = xgb_model.predict(test_dm)
y_test_pred.shape

propensify_test["will respond?"] = y_test_pred

propensify_test.shape

propensify_test["will respond?"] = np.where(propensify_test["will respond?"] > 0.5, 'Yes', 'No')

propensify_test

propensify_test.to_excel('output_fromApp.xlsx', index=False)

C:\Users\debde\AppData\Local\Temp\ipykernel_36320\3327457496.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binaryFromCat_test[col] = encoder.transform(binaryFromCat_test[col])
